In [15]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neural_network import MLPClassifier

In [16]:
# Class to store each tag coordinate data
class tagClass:
    def __init__(self, data: dict, tClass):
        self.x = data.get("x")
        self.y = data.get("y")
        self.fn = data.get("fn").removeprefix("40data/")
        self.label = data.get("label")
        self.tagX, self.tagX2, self.tagY, self.tagY2 = self.getSnip()
        self.tagClass = tClass
    
    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def getTagXY(self):
        return (self.tagX, self.tagY)
    def getTagXY2(self):
        return (self.tagX2, self.tagY2)
    def getFn(self):
        return self.fn
    def getLabel(self):
        return self.label
    def getFileClass(self):
        return tagClass
    
    def getSnip(self):
        newX = self.x-10
        if newX%2 != 0:
            newX -= 1
        newY = self.y-10
        if newY%2 != 0:
            newY -= 1
        return newX, newX+20, newY, newY+20
    
    def getTagCoords(self):
        return self.tagX, self.tagX2, self.tagY, self.tagY2

In [17]:
raw15 = json.load(open("leon_bee_photos_3rdMarch2023/bee_track40_15m.json"))

In [18]:
def getTags(rawData, nType):
    listData = []
    for n in nType:
        for i in rawData[n]:
            i = dict(i)
            d = tagClass(i, n)
            listData.append(d)
    return listData

In [19]:
trainTags = getTags(raw15['0'], ['649', '655', '657', '659', '661'])
testTags = getTags(raw15['0'], ['669', '671'])

In [20]:
bayerFilter = [["R", "G"],
               ["G", "B"]]

In [21]:
# Find Bayer filter pixel colour for given coordinate
def getBayer(x, y):
    if x%2 == 0:
        if y%2 == 0:
            return "R" #RGGB
        else:
            return "G" #GBRG
    else:
        if y%2 == 0:
            return "G" #GRBG
        else:
            return "B" #BGGR

In [22]:
def getTags(l):
    for i in range(20):
        # tagX, tagX2, tagY, tagY2 =
        print(l[i].getTagCoords())
    # photo[tagX:tagX2][tagY:tagY2]

In [23]:
def getSnipPlot(tag):
    tagX, tagX2, tagY, tagY2 = tag.getTagCoords()
    filename = "leon_bee_photos_3rdMarch2023/cam5/"+tag.getFn()
    file = np.load(filename, allow_pickle=True)
    photo = file['img']
    # plt.imshow(photo[tagY:tagY2,tagX:tagX2],cmap='gray')
    # plt.colorbar()
    # rect = patches.Rectangle((tag.getY()-tagY-3.5, tag.getX()-tagX-3.5), 7, 7, linewidth=1, edgecolor='r', facecolor='none')
    # plt.gca().add_patch(rect)
    # plt.show()
    return (tag.getY()-tagY, tag.getX()-tagX), photo

In [24]:
def getPixels(startY, startX, tag: tagClass, photo):
    sx, sy = tag.getTagXY()
    ex, ey = tag.getTagXY2()
    colourPixels = [[], [], []] # R G B
    for px in range(8):
        x = sx + startX + px
        for py in range(8):
            y = sy + startY + py
            col = getBayer(y, x)
            if col == 'R':
                colourPixels[0].append(photo[y, x])
            elif col == 'G':
                colourPixels[1].append(photo[y, x])
            else:
                colourPixels[2].append(photo[y, x])
    totalSum = sum(colourPixels[0]) + sum(colourPixels[1]) + sum(colourPixels[2])
    return sum(colourPixels[0])/totalSum, sum(colourPixels[1])/totalSum

In [25]:
xTags = pd.DataFrame(columns=["Label", "Red", "Green"])
for tag in trainTags:
    search, photo = getSnipPlot(tag)
    redVal, greenVal = getPixels(search[0], search[1], tag, photo)
    xTags.loc[len(xTags.index)] = [int(tag.getLabel().removeprefix("gridTag")), redVal, greenVal]

xTrain, xTest, yTrain, yTest = train_test_split(xTags[["Red", "Green"]], xTags["Label"], test_size=0.2)

In [26]:
yTags = pd.DataFrame(columns=["Label", "Red", "Green"])
for tag in testTags:
    search, photo = getSnipPlot(tag)
    redVal, greenVal = getPixels(search[0], search[1], tag, photo)
    yTags.loc[len(yTags.index)] = [int(tag.getLabel().removeprefix("gridTag")), redVal, greenVal]

In [27]:
gnb = GaussianNB().fit(xTrain, yTrain)
gnbPred = gnb.predict(yTags[["Red", "Green"]])

In [28]:
clf = svm.SVC().fit(xTrain, yTrain)
svmPred = clf.predict(yTags[["Red", "Green"]])

In [29]:
# mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(5, 2), random_state=1).fit(xTrain, yTrain)
# mlpPred = mlp.predict(yTags[["Red", "Green"]])

In [30]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_squared_error
def scores(modelPred, name):
    print(name)
    print("Accuracy: ", accuracy_score(yTags["Label"], modelPred))
    print("F1: ", f1_score(yTags["Label"], modelPred, average='macro'))
    print("Precision: ", precision_score(yTags["Label"], modelPred, average='macro', zero_division=np.nan))
    print("Recall: ", recall_score(yTags["Label"], modelPred, average='macro'))
    print("MSE: ", mean_squared_error(yTags["Label"], modelPred), "\n")

In [31]:
scores(gnbPred, "GNB")
scores(svmPred, "SVM")
# scores(mlpPred, "MLP")

GNB
Accuracy:  0.2125
F1:  0.17331349206349206
Precision:  0.25496715927750413
Recall:  0.2125
MSE:  223.5875 

SVM
Accuracy:  0.1875
F1:  0.12726551226551225
Precision:  0.24620811287477956
Recall:  0.1875
MSE:  298.725 



In [32]:
# average colours from all images - normalise
# R/(R+G+B) G/(R+G+B)
# train model on colours & labels
# test model to classfiy other image labels

# maybe remove background mesh
# func pixel coords generated to show brightness
# psf tagxy brightness pf col, bg of col, pixel ->> pred cixel col -> SSE
# brightness for each col -> then loop all pixels then predict pixels -> SSE = func - true picel val